In [2]:
!pip install accelerate
import accelerate
!pip install sentencepiece
import sentencepiece
!pip install transformers
import transformers
!pip install torch
import torch

**Tokenizer and Model**

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("llama-2.7B-chat")
model = AutoModelForCausalLM.from_pretrained("llama-2.7B-chat",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

**Instruction Based Generation**

In [3]:
system_prompt = """### System:\nInstruction:

**Prompt:** Extract filters from the query and return the result in a structured JSON format. The input query will consist of two parts: a primary query and a filter. The filter will typically be in the format "from [start_date] to [end_date]" or some variation of it. Your task is to:

1. Identify and extract the primary query, which represents the main search topic.
2. Determine the date type (or default "Awarded Date").
3. Extract the start date and end date from the filter, which can be in the form of:
   - A full date (YYYY-MM-DD)
   - A month and year (YYYY-Month), where:
     - The start date should be the first day of the month (YYYY-MM-01).
     - The end date should be the last day of the month (YYYY-MM-last day).
   - A year (YYYY), where:
     - The start date should be the first day of the year (YYYY-01-01).
     - The end date should be the last day of the year (YYYY-12-31).
4. Determine the cost range type (or default "Revised Contract Value").
5. Extract the start amount range and end amount range from the filter, which can include the following:
   - Extract the start amount range and end amount range
   - "under" specifying a start range set to 0 and ending at a specified amount.
   - "above" specifying a start range from a specified amount and end range set to 9999999999.
6.Extract a single or list of UNSPSC Codes, each consisting of 8 digits, from the filter, if present.
7.Extract the "Region" from the from the filter, if present.

Please make sure the JSON format for the response adheres to the following structure:
```
{
  "query": "Primary Query",
  "date_type": "Date_Type",
  "start_date": "Start Date (DD-MM-YYYY)",
  "end_date": "End Date (DD-MM-YYYY)",
  "cost_range_type": "Cost_Range_Type",
  "start_range": "Integer ($)",
  "end_range": "Integer ($)",
  "unspsc_code" : "UNSPSC_Code",
  "region": "Region_Name"
}
```

**Example Usage:** "Extract filters from 'CCTV having Tender Closing Date from 3 Oct 2017 to 13 Jun 2019' having Original Contract Value above $7000 with UNSPSC Code 80101513, 41102614 from region South West"
**Response:**
```
{
  "query": "CCTV",
  "date_type": "Tender Closing Date",
  "start_date": "03-10-2017",
  "end_date": "13-06-2019",
  "cost_range_type": "Original Contract Value",
  "start_range": "7000",
  "end_range": "9999999999",
  "unspsc_code" : [80101513, 41102614],
  "region": "South West"

}
```
\n\n"""

message = "from South West CCTV installation contracts having lease date from Oct 2016 having Supplier contract price cost above $9000 with UNSPSC Code 80101513, 41102614"
prompt = f"{system_prompt}### User: {message}\n\n### Assistant:\n"
prompt_tokens = tokenizer(prompt, return_tensors="pt").to("cuda")["input_ids"]
start_index = prompt_tokens.shape[-1]

output = model.generate(
    prompt_tokens, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=256
)

generation_output = output[0][start_index:]
generation_text = tokenizer.decode(generation_output, skip_special_tokens=True)
print(generation_text)

{
  "query": "from South West CCTV installation contracts having lease date from Oct 2016",
  "date_type": "Lease Date",
  "start_date": "01-10-2016",
  "end_date": null,
  "cost_range_type": "Supplier contract price cost",
  "start_range": "9000",
  "end_range": "9999999999",
  "unspsc_code" : [80101513, 41102614],
  "region": "South West"

}


**Print extracted data**

In [4]:
import json

data = json.loads(generation_text)

print(data)

{'query': 'from South West CCTV installation contracts having lease date from Oct 2016', 'date_type': 'Lease Date', 'start_date': '01-10-2016', 'end_date': None, 'cost_range_type': 'Supplier contract price cost', 'start_range': '9000', 'end_range': '9999999999', 'unspsc_code': [80101513, 41102614], 'region': 'South West'}
